## 기본 라이브러리 불러오기

In [2]:
# 라이브러리 임포트
import os
import pandas as pd
import chardet

# 사용자 정의함수 경로 설정
import sys
sys.path.append("G:/내 드라이브/Source/_Custom_Function")
# 사용자 정의함수 불러오기

import time

import ExcelFile_Merge as em
import ExcelFile_Read as er
import ExcelFile_EDA as ee

## 기본 데이터 불러오기

### 서울특별시 공공자전거 대여이력 정보

In [30]:
start_time = time.time()
df_BikeRent_2023 = pd.read_parquet('G:\\내 드라이브\\DataSet\\_파킷 파일\\서울특별시 공공자전거 대여이력 정보\\서울특별시 공공자전거 대여이력 정보_2023_preprocessed.parquet')
print("read_csv time :", time.time() - start_time)

read_csv time : 29.21795654296875


In [ ]:
start_time = time.time()
df_BikeRent_2024 = pd.read_parquet('G:\\내 드라이브\\DataSet\\_파킷 파일\\서울특별시 공공자전거 대여이력 정보\\서울특별시 공공자전거 대여이력 정보_2024_preprocessed.parquet')
print("read_csv time :", time.time() - start_time)

In [ ]:
df_BikeRent = pd.concat([df_BikeRent_2023, df_BikeRent_2024], axis=0)

In [32]:
df_BikeRent.head(2)

,자전거번호,대여대여소번호,대여대여소명,대여거치대,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M),대여일자,대여시간,반납일자,반납시간,대여시간차,이용시간-대여시간차,이용속도(km/h)
0,SPB-46712,04804,동서울농협 앞,0,01416,상봉역 3번출구,0,1,0.00,2024-01-01,00:04:10,2024-01-01,00:05:13,0 days 00:01:03,-0.050000,0.0000
1,SPB-35089,04169,신대방삼거리,0,00251,서울지방병무청 버스정류장,0,6,1147.58,2024-01-01,00:00:10,2024-01-01,00:06:23,0 days 00:06:13,-0.216667,11.4758
2,SPB-36386,00540,군자역 7번출구 베스트샵 앞,0,03509,세종사이버대학교,0,3,642.13,2024-01-01,00:03:13,2024-01-01,00:06:33,0 days 00:03:20,-0.333333,12.8426


In [34]:
df_BikeRent.shape[0]

21621807

In [40]:
# 대여일자 : 2024-01-01 ~ 2024-06-30
df_BikeRent.sort_values(by = '대여일자', ascending = True).head(2)

,자전거번호,대여대여소번호,대여대여소명,대여거치대,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M),대여일자,대여시간,반납일자,반납시간,대여시간차,이용시간-대여시간차,이용속도(km/h)
0,SPB-46712,04804,동서울농협 앞,0,01416,상봉역 3번출구,0,1,0.0,2024-01-01,00:04:10,2024-01-01,00:05:13,0 days 00:01:03,-0.05,0.000000
32595,SPB-64831,02723,마곡 필네이처,0,02735,마곡 공공청사,0,9,1090.0,2024-01-01,17:32:35,2024-01-01,17:41:41,0 days 00:09:06,-0.10,7.266667


### 서울시 공공자전거 고장신고 내역

In [10]:
start_time = time.time()
df_BikeBroken = pd.read_parquet('G:\\내 드라이브\\DataSet\\_최종 병합 파일\\서울시 공공자전거 고장신고 내역_15-24.06.parquet')
print("read_csv time :", time.time() - start_time)

read_csv time : 1.7008225917816162


In [13]:
df_BikeBroken.head(3)

,자전거번호,고장구분,등록일자,등록시간
0,SPB-00108,체인,2015-12-04,07:41:01
1,SPB-00210,체인,2015-12-07,12:23:33
2,SPB-00035,기타,2015-12-09,17:07:00


In [15]:
df_BikeBroken.shape[0]

767715

---
### df_BikeRent 의 데이터를 조건에 재조회
- 대여시간차가 5분 미만인 데이터
- 대여 대여소 번호와 반납 대여소 번호가 동일한 데이터

In [20]:
# 1. 조건에 따라 필터링: 대여시간차가 5분 미만, 대여 대여소 번호와 반납 대여소 번호가 동일한 데이터만 선택
cond1 = df_BikeRent['대여시간차'] < pd.Timedelta(minutes=5)
cond2 = df_BikeRent['대여대여소번호'] == df_BikeRent['반납대여소번호']
df_BikeRent_2 = df_BikeRent[cond1 & cond2].copy()

# 대여일자를 날짜 형식으로 변환 (필요한 경우)
df_BikeRent_2['대여일자'] = pd.to_datetime(df_BikeRent_2['대여일자'])

In [21]:
df_BikeRent_2.shape[0]

1540797

In [24]:
df_BikeRent_2.head(3)

,자전거번호,대여대여소번호,대여대여소명,대여거치대,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M),대여일자,대여시간,반납일자,반납시간,대여시간차,이용시간-대여시간차,이용속도(km/h)
0,SPB-39194,01554,번동사거리,0,01554,번동사거리,0,0,0.0,2023-01-01,00:02:13,2023-01-01,00:02:43,0 days 00:00:30,-0.500000,NaN
2,SPB-44383,02601,석촌호수 아뜰리에,0,02601,석촌호수 아뜰리에,0,0,0.0,2023-01-01,00:04:01,2023-01-01,00:04:21,0 days 00:00:20,-0.333333,NaN
10,SPB-64954,02703,서울도시가스 앞,0,02703,서울도시가스 앞,0,1,0.0,2023-01-01,00:05:49,2023-01-01,00:06:26,0 days 00:00:37,0.383333,0.0


---
1. 2024년 고장신고내역의 자전거번호를 구한다.
2. 2024년 자전거대여이력의 자전거번호를 구한다.
3. 

In [48]:
df_BikeBroken.head()

,자전거번호,고장구분,등록일자,등록시간
0,SPB-00108,체인,2015-12-04,07:41:01
1,SPB-00210,체인,2015-12-07,12:23:33
2,SPB-00035,기타,2015-12-09,17:07:00
3,SPB-01024,기타,2015-12-13,21:02:24
4,SPB-01015,기타,2015-12-28,10:12:04


In [60]:
df_BikeBroken.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 767715 entries, 0 to 767714
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   자전거번호   767715 non-null  string
 1   고장구분    767715 non-null  string
 2   등록일자    767715 non-null  string
 3   등록시간    618185 non-null  string
dtypes: string(4)
memory usage: 23.4 MB


---
### 2024년 고장이력만 조회
- 등록일자 : 2024-01-01 ~ 2024-06-30

In [63]:
df_BikeBroken2 = df_BikeBroken[df_BikeBroken['등록일자'].between('2024-01-01', '2024-12-31')]

In [65]:
print(f'df_BikeBroken.shape[0] {df_BikeBroken.shape[0]}')
print(f'df_BikeBroken2.shape[0] {df_BikeBroken2.shape[0]}')

df_BikeBroken.shape[0] 767715
df_BikeBroken2.shape[0] 76463


In [58]:
df_BikeBroken2['자전거번호'].unique()

<StringArray>
['SPB-40704', 'SPB-65171', 'SPB-44316', 'SPB-31539', 'SPB-51998', 'SPB-38568',
 'SPB-68235', 'SPB-44078', 'SPB-66158', 'SPB-45102',
 ...
 'SPB-64200', 'SPB-42922', 'SPB-69655', 'SPB-39650', 'SPB-38653', 'SPB-62989',
 'SPB-31854', 'SPB-62325', 'SPB-62654', 'SPB-35527']
Length: 28589, dtype: string

In [69]:
df_BikeBroken2['자전거번호'].value_counts()

자전거번호
SPB-30296    24
SPB-70298    23
SPB-31172    21
SPB-41406    21
SPB-33626    19
             ..
SPB-60257     1
SPB-40210     1
SPB-59161     1
SPB-32403     1
SPB-35527     1
Name: count, Length: 28589, dtype: Int64

In [88]:
df_BikeBroken2.sort_values(by = '등록일자', ascending = False).head(2)

,자전거번호,고장구분,등록일자,등록시간
767714,SPB-62036,안장,2024-06-30,23:58:29
767536,SPB-64227,안장,2024-06-30,16:03:40


In [77]:
df_BikeBroken2[df_BikeBroken2['자전거번호'] == 'SPB-30296'].sort_values(by = '등록일자', ascending = False).head(2)

,자전거번호,고장구분,등록일자,등록시간
752797,SPB-30296,기타,2024-06-06,14:51:27
752796,SPB-30296,체인,2024-06-06,14:51:27


---
### 자전거고장신고내역에서
- 자전거번호' 별로 가장마지막 신고내역만 가져오기

In [114]:
# 등록일자를 datetime 형식으로 변환
df_BikeBroken2['등록일자'] = pd.to_datetime(df_BikeBroken2['등록일자'])

# 자전거번호 별로 가장 최근 등록일자를 가진 행 추출
df_BikeBroken_latest = df_BikeBroken2.loc[df_BikeBroken2.groupby('자전거번호')['등록일자'].idxmax()]

# 이전일자 컬럼 생성 (등록일자보다 한 달 전)
df_BikeBroken_latest['이전일자'] = df_BikeBroken_latest['등록일자'] - pd.DateOffset(months=1)

# 결과 출력
df_BikeBroken_latest.head(2)

C:\Users\user\AppData\Local\Temp\ipykernel_6888\2516430342.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_BikeBroken2['등록일자'] = pd.to_datetime(df_BikeBroken2['등록일자'])


,자전거번호,고장구분,등록일자,등록시간,이전일자
714016,SPB-30006,페달,2024-03-31,23:48:49,2024-02-29
760656,SPB-30010,타이어,2024-06-18,18:37:09,2024-05-18


In [116]:
df_BikeBroken_latest = df_BikeBroken_latest[['자전거번호', '고장구분', '이전일자', '등록일자']]

In [118]:
df_BikeBroken_latest.head(2)

,자전거번호,고장구분,이전일자,등록일자
714016,SPB-30006,페달,2024-02-29,2024-03-31
760656,SPB-30010,타이어,2024-05-18,2024-06-18


#### 고장난 자전기 가져오기

In [102]:
df_BikeBroken_latest['자전거번호'].unique()

<StringArray>
['SPB-30006', 'SPB-30010', 'SPB-30014', 'SPB-30018', 'SPB-30022', 'SPB-30024',
 'SPB-30026', 'SPB-30031', 'SPB-30036', 'SPB-30037',
 ...
 'SPB-84955', 'SPB-84963', 'SPB-84968', 'SPB-84974', 'SPB-84984', 'SPB-84988',
 'SPB-84989', 'SPB-84993', 'SPB-84995', 'SPB-84996']
Length: 28589, dtype: string

In [99]:
df_BikeBroken_latest.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28589 entries, 714016 to 737981
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   자전거번호   28589 non-null  string        
 1   고장구분    28589 non-null  string        
 2   등록일자    28589 non-null  datetime64[ns]
 3   등록시간    28589 non-null  string        
 4   이전일자    28589 non-null  datetime64[ns]
dtypes: datetime64[ns](2), string(3)
memory usage: 1.3 MB


In [108]:
df_BikeRent2 = df_BikeRent[~df_BikeRent['자전거번호'].isin(df_BikeBroken_latest['자전거번호'].unique())]

In [112]:
df_BikeRent2['자전거번호'].count()

4855582

In [120]:
df_BikeRent2.head(2)

,자전거번호,대여대여소번호,대여대여소명,대여거치대,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M),대여일자,대여시간,반납일자,반납시간,대여시간차,이용시간-대여시간차,이용속도(km/h)
8,SPB-80304,00214,금융감독원 앞,99,00213,KT앞,99,1,200.00,2024-01-01,00:06:18,2024-01-01,00:08:13,0 days 00:01:55,-0.916667,12.0000
22,SPB-32497,04674,녹번역e편한세상 캐슬 정문 앞,0,04659,신진과학기술고등학교 버스정류장(응암시장 방면),0,5,982.45,2024-01-01,00:04:32,2024-01-01,00:09:32,0 days 00:05:00,0.000000,11.7894


In [122]:
# 등록일자를 datetime 형식으로 변환
df_BikeRent2['대여일자'] = pd.to_datetime(df_BikeRent2['대여일자'])

# 자전거번호 별로 가장 최근 등록일자를 가진 행 추출
df_BikeRent_latest = df_BikeRent2.loc[df_BikeRent2.groupby('자전거번호')['대여일자'].idxmax()]

# 이전일자 컬럼 생성 (등록일자보다 한 달 전)
df_BikeRent_latest['이전일자'] = df_BikeRent_latest['대여일자'] - pd.DateOffset(months=1)

C:\Users\user\AppData\Local\Temp\ipykernel_6888\2407400792.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_BikeRent2['대여일자'] = pd.to_datetime(df_BikeRent2['대여일자'])


In [124]:
df_BikeRent_latest.head(2)

,자전거번호,대여대여소번호,대여대여소명,대여거치대,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M),대여일자,대여시간,반납일자,반납시간,대여시간차,이용시간-대여시간차,이용속도(km/h),이전일자
21756509,SPB-30001,03798,그랜드아이파크아파트 앞,0,01174,강서구청사거리(부민병원),0,1,0.00,2024-06-28,08:25:40,2024-06-28,08:27:15,0 days 00:01:35,-0.583333,0.000000,2024-05-28
14976293,SPB-30008,03920,개봉역2번출구 B,0,00785,"양천구청, 보건소 사잇길",0,21,3419.51,2024-05-20,08:19:08,2024-05-20,08:40:29,0 days 00:21:21,-0.350000,9.770029,2024-04-20


In [126]:
df_BikeRent_latest.shape[0]

13307

### 대여이력 <=> 고장내역 - 컬럼이름 맞춰주기

In [132]:
df_BikeBroken_latest.columns

Index(['자전거번호', '고장구분', '이전일자', '등록일자'], dtype='object')

In [135]:
df_BikeRent_latest.columns

Index(['자전거번호', '대여대여소번호', '대여대여소명', '대여거치대', '반납대여소번호', '반납대여소명', '반납거치대',
       '이용시간(분)', '이용거리(M)', '대여일자', '대여시간', '반납일자', '반납시간', '대여시간차',
       '이용시간-대여시간차', '이용속도(km/h)', '이전일자', '고장구분'],
      dtype='object')

In [128]:
df_BikeRent_latest['고장구분'] = ''

In [138]:
df_BikeBroken_latest.rename(columns = {'등록일자':'등록대여일자'}, inplace = True)
df_BikeRent_latest.rename(columns = {'대여일자':'등록대여일자'}, inplace = True)

In [142]:
df_BikeBroken_latest = df_BikeBroken_latest[['자전거번호', '고장구분', '이전일자', '등록대여일자']]
df_BikeRent_latest = df_BikeRent_latest[['자전거번호', '고장구분', '이전일자', '등록대여일자']]

In [144]:
df_BikeBroken_latest.head(2)

,자전거번호,고장구분,이전일자,등록대여일자
714016,SPB-30006,페달,2024-02-29,2024-03-31
760656,SPB-30010,타이어,2024-05-18,2024-06-18


In [148]:
df_BikeBroken_latest['고장여부'] = 1
df_BikeRent_latest['고장여부'] = 0

C:\Users\user\AppData\Local\Temp\ipykernel_6888\161305037.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_BikeRent_latest['고장여부'] = 0


In [150]:
df_BikeRent_latest.head(2)

,자전거번호,고장구분,이전일자,등록대여일자,고장여부
21756509,SPB-30001,,2024-05-28,2024-06-28,0
14976293,SPB-30008,,2024-04-20,2024-05-20,0


---
### 자전거(고장) - 자전거(일반) 합치기

In [153]:
df_Bike_latest = pd.concat([df_BikeBroken_latest, df_BikeRent_latest], axis=0)

In [157]:
print(f'df_BikeBroken_latest.shape[0] : {df_BikeBroken_latest.shape[0]}')
print(f'df_BikeRent_latest.shape[0] : {df_BikeRent_latest.shape[0]}')
print(f'df_Bike_latest.shape[0] : {df_Bike_latest.shape[0]}')

df_BikeBroken_latest.shape[0] : 28589
df_BikeRent_latest.shape[0] : 13307
df_Bike_latest.shape[0] : 41896


In [161]:
df_Bike_latest.head()

,자전거번호,고장구분,이전일자,등록대여일자,고장여부
714016,SPB-30006,페달,2024-02-29,2024-03-31,1
760656,SPB-30010,타이어,2024-05-18,2024-06-18,1
763779,SPB-30014,기타,2024-05-24,2024-06-24,1
749636,SPB-30018,기타,2024-05-01,2024-06-01,1
741124,SPB-30022,기타,2024-04-20,2024-05-20,1


In [163]:
df_BikeRent.head()

,자전거번호,대여대여소번호,대여대여소명,대여거치대,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M),대여일자,대여시간,반납일자,반납시간,대여시간차,이용시간-대여시간차,이용속도(km/h)
0,SPB-46712,04804,동서울농협 앞,0,01416,상봉역 3번출구,0,1,0.00,2024-01-01,00:04:10,2024-01-01,00:05:13,0 days 00:01:03,-0.050000,0.0000
1,SPB-35089,04169,신대방삼거리,0,00251,서울지방병무청 버스정류장,0,6,1147.58,2024-01-01,00:00:10,2024-01-01,00:06:23,0 days 00:06:13,-0.216667,11.4758
2,SPB-36386,00540,군자역 7번출구 베스트샵 앞,0,03509,세종사이버대학교,0,3,642.13,2024-01-01,00:03:13,2024-01-01,00:06:33,0 days 00:03:20,-0.333333,12.8426
3,SPB-61278,01139,용문사 버스정류장,0,00701,목동사거리 부근,0,4,830.00,2024-01-01,00:02:42,2024-01-01,00:07:24,0 days 00:04:42,-0.700000,12.4500
4,SPB-36848,03416,동묘앞역 6번출구,0,00346,맥스타일 앞,0,4,791.08,2024-01-01,00:03:02,2024-01-01,00:07:33,0 days 00:04:31,-0.516667,11.8662


---
### '자전거고장신고내역'의 '이전일자' ~ '등록대여일자'가 '자전거대여이력'의 '대여일자'에 포함되는 자전거별 횟수 가져오기

In [166]:
start_time = time.time()

# 결과를 저장할 딕셔너리 초기화
results = {}

# 고장 신고 내역을 순회하며 대여 이력에서 해당 기간에 속하는 대여 건수를 카운트
for index, row in df_Bike_latest.iterrows():
    bike_number = row['자전거번호']
    previous_date = row['이전일자']
    registered_date = row['등록대여일자']

    # 자전거 번호가 일치하고, 대여일자가 이전일자와 등록대여일자 사이에 있는 경우의 수를 카운트
    rental_count = df_BikeRent_2[(df_BikeRent_2['자전거번호'] == bike_number) & 
                               (df_BikeRent_2['대여일자'] >= previous_date) & 
                               (df_BikeRent_2['대여일자'] <= registered_date)].shape[0]

    # 결과를 딕셔너리에 저장
    results[bike_number] = rental_count

print("check time :", time.time() - start_time)

check time : 3792.225153207779


In [ ]:
start_time = time.time()

# 결과를 저장할 딕셔너리 초기화
results = []

# 고장 신고 내역을 순회하며 대여 이력에서 해당 기간에 속하는 대여 건수를 카운트
for index, row in df_Bike_latest.iterrows():
    bike_number = row['자전거번호']
    previous_date = row['이전일자']
    registered_date = row['등록대여일자']

    # 자전거 번호가 일치하고, 대여일자가 이전일자와 등록대여일자 사이에 있는 경우의 수를 카운트
    rental_count = df_BikeRent_2[(df_BikeRent_2['자전거번호'] == bike_number) & 
                               (df_BikeRent_2['대여일자'] >= previous_date) & 
                               (df_BikeRent_2['대여일자'] <= registered_date)].shape[0]

    results.append({
            '자전거번호': bike_number,
            '이전일자': previous_date,
            '등록대여일자': registered_date,
            '고장구분': row['고장구분'],
            '고장여부': row['고장여부'],
            '고장횟수': rental_count,
        })

# DataFrame으로 변환
df_bike_results = pd.DataFrame(results)

print("check time :", time.time() - start_time)

In [168]:
print(results)

{'SPB-30006': 0, 'SPB-30010': 0, 'SPB-30014': 0, 'SPB-30018': 0, 'SPB-30022': 0, 'SPB-30024': 0, 'SPB-30026': 0, 'SPB-30031': 0, 'SPB-30036': 0, 'SPB-30037': 0, 'SPB-30041': 0, 'SPB-30044': 0, 'SPB-30045': 0, 'SPB-30052': 0, 'SPB-30053': 0, 'SPB-30057': 0, 'SPB-30061': 0, 'SPB-30062': 0, 'SPB-30070': 0, 'SPB-30071': 0, 'SPB-30074': 0, 'SPB-30076': 0, 'SPB-30077': 0, 'SPB-30078': 0, 'SPB-30080': 0, 'SPB-30091': 0, 'SPB-30095': 0, 'SPB-30096': 0, 'SPB-30098': 0, 'SPB-30099': 0, 'SPB-30101': 0, 'SPB-30103': 0, 'SPB-30104': 0, 'SPB-30105': 0, 'SPB-30107': 0, 'SPB-30108': 0, 'SPB-30111': 0, 'SPB-30112': 0, 'SPB-30114': 0, 'SPB-30115': 0, 'SPB-30117': 0, 'SPB-30118': 0, 'SPB-30119': 0, 'SPB-30120': 0, 'SPB-30122': 0, 'SPB-30126': 0, 'SPB-30127': 0, 'SPB-30128': 0, 'SPB-30129': 0, 'SPB-30130': 0, 'SPB-30131': 0, 'SPB-30132': 0, 'SPB-30133': 0, 'SPB-30134': 0, 'SPB-30135': 0, 'SPB-30136': 0, 'SPB-30137': 0, 'SPB-30140': 0, 'SPB-30141': 0, 'SPB-30145': 0, 'SPB-30147': 0, 'SPB-30148': 0, 'SPB-30

In [172]:
len(results)

41896

In [178]:
type(results)

dict

In [180]:
df_bike_results = pd.DataFrame(list(results.items()), columns=['자전거번호', '고장횟수'])

In [182]:
df_bike_results.head()

,자전거번호,고장횟수
0,SPB-30006,0
1,SPB-30010,0
2,SPB-30014,0
3,SPB-30018,0
4,SPB-30022,0


In [192]:
df_bike_results[df_bike_results['고장횟수'] > 0].head(2)

,자전거번호,고장횟수
118,SPB-30243,1
123,SPB-30251,1


In [186]:
df_Bike_latest.head(2)

,자전거번호,고장구분,이전일자,등록대여일자,고장여부
714016,SPB-30006,페달,2024-02-29,2024-03-31,1
760656,SPB-30010,타이어,2024-05-18,2024-06-18,1


In [188]:
df_Bike_latest_2 = pd.merge(df_Bike_latest, df_bike_results, on='자전거번호', how='outer')

In [190]:
df_Bike_latest_2.head(2)

,자전거번호,고장구분,이전일자,등록대여일자,고장여부,고장횟수
0,SPB-30001,,2024-05-28,2024-06-28,0,0
1,SPB-30006,페달,2024-02-29,2024-03-31,1,0


In [194]:
df_Bike_latest_2[df_Bike_latest_2['고장횟수'] > 0]

,자전거번호,고장구분,이전일자,등록대여일자,고장여부,고장횟수
184,SPB-30243,타이어,2023-12-13,2024-01-13,1,1
185,SPB-30244,,2023-12-18,2024-01-18,0,1
192,SPB-30251,기타,2023-12-10,2024-01-10,1,1
308,SPB-30374,페달,2023-12-19,2024-01-19,1,1
537,SPB-30614,체인,2023-12-24,2024-01-24,1,2
...,...,...,...,...,...,...
41366,SPB-84449,타이어,2023-12-01,2024-01-01,1,5
41615,SPB-84708,타이어,2023-12-05,2024-01-05,1,1
41618,SPB-84711,기타,2023-12-14,2024-01-14,1,1
41674,SPB-84769,체인,2023-12-04,2024-01-04,1,1


---
## 고장횟수 별 - 고장비율

### [전체] 고장여부 비율

In [240]:
counts = df_Bike_latest_2['고장여부'].value_counts()
total = counts.sum()   # 총 데이터 개수
percentage = counts / total * 100   # 비율 계산
print(f'total : {total}')
print(f'counts : {counts}')
print(f'percentage : {percentage}')

total : 41896
counts : 고장여부
1    28589
0    13307
Name: count, dtype: int64
percentage : 고장여부
1    68.238018
0    31.761982
Name: count, dtype: float64


### [ 고장횟수 == 0 ] 것들의 고장여부 비율

In [238]:
counts = df_Bike_latest_2[df_Bike_latest_2['고장횟수'] == 0]['고장여부'].value_counts()
total = counts.sum()   # 총 데이터 개수
percentage = counts / total * 100   # 비율 계산
print(f'total : {total}')
print(f'counts : {counts}')
print(f'percentage : {percentage}')

total : 41243
counts : 고장여부
1    27976
0    13267
Name: count, dtype: int64
percentage : 고장여부
1    67.832117
0    32.167883
Name: count, dtype: float64


### [ 고장횟수 > 0 ] 것들의 고장여부 비율

In [242]:
counts = df_Bike_latest_2[df_Bike_latest_2['고장횟수'] > 0]['고장여부'].value_counts()
total = counts.sum()   # 총 데이터 개수
percentage = counts / total * 100   # 비율 계산
print(f'total : {total}')
print(f'counts : {counts}')
print(f'percentage : {percentage}')

total : 653
counts : 고장여부
1    613
0     40
Name: count, dtype: int64
percentage : 고장여부
1    93.874426
0     6.125574
Name: count, dtype: float64


### [ 고장횟수 > 1 ] 것들의 고장여부 비율

In [244]:
counts = df_Bike_latest_2[df_Bike_latest_2['고장횟수'] > 1]['고장여부'].value_counts()
total = counts.sum()   # 총 데이터 개수
percentage = counts / total * 100   # 비율 계산
print(f'total : {total}')
print(f'counts : {counts}')
print(f'percentage : {percentage}')

total : 362
counts : 고장여부
1    342
0     20
Name: count, dtype: int64
percentage : 고장여부
1    94.475138
0     5.524862
Name: count, dtype: float64


### [ 고장횟수 > 2 ] 것들의 고장여부 비율

In [247]:
counts = df_Bike_latest_2[df_Bike_latest_2['고장횟수'] > 2]['고장여부'].value_counts()
total = counts.sum()   # 총 데이터 개수
percentage = counts / total * 100   # 비율 계산
print(f'total : {total}')
print(f'counts : {counts}')
print(f'percentage : {percentage}')

total : 214
counts : 고장여부
1    206
0      8
Name: count, dtype: int64
percentage : 고장여부
1    96.261682
0     3.738318
Name: count, dtype: float64
